In [37]:
import torch
import torch.nn as nn
from torch.nn import Conv2d, MaxPool2d, Linear, Flatten
import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import time
from model import MyModule

# 0. 设置设备（自动检测 CUDA / MPS / CPU）
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using devic e: {device}")

# 1. 准备数据集
train_data = datasets.CIFAR10(
    root="./dataset",
    transform=transforms.ToTensor(),
    train=True,
    download=True,
)

test_data = datasets.CIFAR10(
    root="./dataset",
    transform=transforms.ToTensor(),
    train=False,
    download=True,
)

train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"train_Dataset size: {train_data_size}")
print(f"test_Dataset size: {test_data_size}")

# 3.定义DataLoader
# dataloader 加载数据集
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=False,
)


# 4.实例化模型

myModule = MyModule().to(device)

# print(myModule)

# 5.定义损失函数
loss_fn = nn.CrossEntropyLoss().to(device)


# 6.优化器
# 1e-2= 1*(10)^-2 = 0.01
learning_rate = 1e-3
optimizer = torch.optim.Adam(myModule.parameters(), lr=learning_rate)


# 7.训练模型
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

write = SummaryWriter("logs")
# 训练
for i in range(epoch):

    # train mode 的作用: 将网络设置为训练模式
    myModule.train()

    # 7.1训练模型
    print(f"----------第 {i+1} 轮训练开始----------")
    for data in train_loader:
        imgs, targets = data
        # 将数据拷贝到device上
        imgs = imgs.to(device)
        targets = targets.to(device)

        # 前向传播
        outputs = myModule(imgs)
        loss = loss_fn(outputs, targets)

        # 反向传播
        # 优化器优化模型
        # 梯度清零
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 记录训练损失
        total_train_step += 1
        if total_train_step % 100 == 0:
            print(f"训练次数: {total_train_step}, Loss: {loss.item():.3f}")
            write.add_scalar("train_loss", loss.item(), total_train_step)

    # eval mode 的作用: 将网络设置为测试模式
    myModule.eval()
    # 7.2测试模型
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_loader:
            imgs, targets = data
            # 将数据拷贝到device上
            imgs = imgs.to(device)
            targets = targets.to(device)

            outputs = myModule(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            correct = (outputs.argmax(1) == targets).sum().item()
            total_accuracy += correct
        total_test_step += 1
        print(
            f"测试次数: {total_test_step}, 测试集的Loss: {total_test_loss:.3f}, 测试集的准确率: {total_accuracy/test_data_size:.2%}"
        )
        write.add_scalar("test_loss", total_test_loss, total_test_step)

    # 7.3保存模型
    torch.save(myModule.state_dict(), f"./myModule_{i+1}.pth")  # ✅ 仅保存参数
write.close()

Using devic e: mps
Files already downloaded and verified
Files already downloaded and verified
train_Dataset size: 50000
test_Dataset size: 10000
----------第 1 轮训练开始----------
训练次数: 100, Loss: 1.668
训练次数: 200, Loss: 1.660
训练次数: 300, Loss: 1.436
训练次数: 400, Loss: 1.509
训练次数: 500, Loss: 1.514
训练次数: 600, Loss: 1.078
训练次数: 700, Loss: 1.099
测试次数: 1, 测试集的Loss: 193.924, 测试集的准确率: 56.88%
----------第 2 轮训练开始----------
训练次数: 800, Loss: 1.086
训练次数: 900, Loss: 1.065
训练次数: 1000, Loss: 1.590
训练次数: 1100, Loss: 0.886
训练次数: 1200, Loss: 1.070
训练次数: 1300, Loss: 1.033
训练次数: 1400, Loss: 0.854
训练次数: 1500, Loss: 1.343
测试次数: 2, 测试集的Loss: 166.041, 测试集的准确率: 63.39%
----------第 3 轮训练开始----------
训练次数: 1600, Loss: 0.690
训练次数: 1700, Loss: 0.843
训练次数: 1800, Loss: 0.998
训练次数: 1900, Loss: 1.033
训练次数: 2000, Loss: 0.775
训练次数: 2100, Loss: 0.922
训练次数: 2200, Loss: 1.118
训练次数: 2300, Loss: 0.899
测试次数: 3, 测试集的Loss: 153.134, 测试集的准确率: 67.26%
----------第 4 轮训练开始----------
训练次数: 2400, Loss: 0.977
训练次数: 2500, Loss: 0.657
训练次数: 2600,

In [27]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else 
    "mps" if torch.backends.mps.is_available() else 
    "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [12]:
import os
import platform

# CPU 型号
print(platform.processor())

# 核心数
print(os.cpu_count())  # 逻辑核心数

arm
8


In [13]:
from Foundation import NSBundle
import subprocess

# 调用系统命令获取 GPU 信息
gpu_info = subprocess.check_output(["system_profiler", "SPDisplaysDataType"]).decode("utf-8")
print(gpu_info)

Graphics/Displays:

    Apple M1 Pro:

      Chipset Model: Apple M1 Pro
      Type: GPU
      Bus: Built-In
      Total Number of Cores: 14
      Vendor: Apple (0x106b)
      Metal Support: Metal 3
      Displays:
        U2790B:
          Resolution: 3840 x 2160 (2160p/4K UHD 1 - Ultra High Definition)
          UI Looks like: 1920 x 1080 @ 60.00Hz
          Main Display: Yes
          Mirror: Off
          Online: Yes
          Rotation: Supported
        Color LCD:
          Display Type: Built-in Liquid Retina XDR Display
          Resolution: 3024 x 1964 Retina
          Mirror: Off
          Online: Yes
          Automatically Adjust Brightness: No
          Connection Type: Internal


